In [197]:
from llama_index import LLMPredictor, ServiceContext
# from llama_index import VectorStoreIndex
# from llama_index import SimpleDirectoryReader
from llama_index import Prompt
from llama_index import PromptHelper
from llama_index import StorageContext, load_index_from_storage
from llama_index.evaluation import ResponseEvaluator
from llama_index.llms import OpenAI
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index.indices.query.query_transform import HyDEQueryTransform
from llama_index.query_engine.transform_query_engine import TransformQueryEngine
from langchain.chat_models import ChatOpenAI
from IPython.display import Markdown, display

import environ
import openai

In [198]:
selected_llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [199]:
# For now I use my key
env = environ.Env()
environ.Env.read_env()
API_KEY = env("OPENAI_API_KEY")
openai.api_key = API_KEY

/home/daniele/Desktop/Projects/bureaubot/.venv_bureaubot/lib/python3.10/site-packages/environ/environ.py:639: UserWarning: /tmp/ipykernel_9482/.env doesn't exist - if you're not configuring your environment separately, create one.
  warnings.warn(


# Load vector store

In [200]:
embedding_path = "vector_db"
# rebuild storage context
storage_context_from_load = StorageContext.from_defaults(persist_dir=embedding_path)
# load index
index_loaded = load_index_from_storage(storage_context_from_load)

# Functions

In [201]:
# Evaluator
def eval_response(response):
    llm= LLMPredictor(llm=selected_llm)
    service_context_eval = ServiceContext.from_defaults(llm_predictor=llm)
    evaluator = ResponseEvaluator(service_context=service_context_eval)
    eval_result = evaluator.evaluate(response)
    if eval_result == "NO":
        print(str(eval_result))
        print("info not available")
        return None
    elif eval_result == "YES":
        display(Markdown(response.response))
        response_metadata_chat = dict()
        for i, source_node in enumerate(response.source_nodes):
            key_name = "ref_" + str(i)
            response_metadata_chat[key_name] = {
                "page": source_node.node.metadata["page_label"],
                "document":source_node.node.metadata["file_name"]
            }
        return response_metadata_chat
    else:
        print("something went wrong, try again!")
        return None

In [202]:
def print_response_info(user_query, response):
    print(f"Prompt:\n{user_query}")
    display(Markdown(f"Response:<br/>{response.response}"))
    print(f"Sources:")
    for node in response.source_nodes:
        print(f'Page: {node.node.metadata["page_label"]}, Document: {node.node.metadata["file_name"]}, Cosine Similarity: {node.score:.2f}')

# Define query and prompt template, folders and top_k

In [203]:
# user_query = "Is the number of children taken into account to calculate the unemployment benefit?"
# user_query = "What do you know about the city of Berlin?" # check for generic responses from GPT knowledge base
user_query = "Wann muss ich mich melden wenn ich Arbeitslosengeld beantragen will?"

# Define prompt
template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Do not give me an answer if it is not mentioned in the context as a fact. \n"
    "Given this information, please provide me with an answer to the following question:\n{query_str}\n"
)
qa_template = Prompt(template)


folder_with_index = "vector_db"
number_top_results = 5 # Number of top results to return

# Query Engine

## Simple query engine

In [204]:
query_engine_baseline = index_loaded.as_query_engine(text_qa_template=qa_template, similarity_top_k=number_top_results)
response_baseline = query_engine_baseline.query(user_query)

print_response_info(user_query, response_baseline)

Prompt:
Wann muss ich mich melden wenn ich Arbeitslosengeld beantragen will?


Response:<br/>Sie müssen sich spätestens am ersten Tag Ihrer Arbeitslosigkeit arbeitslos melden, um Arbeitslosengeld zu beantragen.

Sources:
Page: 15, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.89
Page: 6, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.89
Page: 13, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.89
Page: 14, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.88
Page: 18, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.88


In [205]:
for uwdfujsn in response_baseline.source_nodes:
    print(uwdfujsn.node.metadata)

{'page_label': '15', 'file_name': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}
{'page_label': '6', 'file_name': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}
{'page_label': '13', 'file_name': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}
{'page_label': '14', 'file_name': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}
{'page_label': '18', 'file_name': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}


In [206]:
response_text = response_baseline.response
response_metadata = dict()
response_metadata_message = f'There {len(response_baseline.metadata)} sources:'
for i, meta_data in enumerate(response_baseline.metadata):
    key_name = "ref_" + str(i)
    response_metadata[key_name] = {
        "page": response_baseline.metadata[meta_data]["page_label"],
        "document":response_baseline.metadata[meta_data]["file_name"]
    }
    response_metadata_message += "\n---" + key_name + f'Page {response_baseline.metadata[meta_data]["page_label"]} from file {response_baseline.metadata[meta_data]["file_name"]}'

In [207]:
response_baseline

Response(response='Sie müssen sich spätestens am ersten Tag Ihrer Arbeitslosigkeit arbeitslos melden, um Arbeitslosengeld zu beantragen.', source_nodes=[NodeWithScore(node=TextNode(id_='c0ed0f94-d086-406a-b19d-0a69f1370419', embedding=None, metadata={'page_label': '15', 'file_name': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7e18153e-10e3-42bf-8a3a-6463eaa5602a', node_type=None, metadata={'page_label': '15', 'file_name': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}, hash='a828cbb7c8c6afd100be42c39f363a3e2414e8b9b716def616214a4002610a8f')}, hash='e7e24f5977eb93ef3fbb97e9e27da653863f028f8007306d8747d5702d5a9280', text='2. Was müssen Sie tun, wenn Arbeitslosigkeit eintritt?\n16Wenn Sie sich nicht am ersten Tag nach dem Ende \n Ihrer Beschäftigung online oder persönlich  arbeitslos \nmelden  können, weil Ihre Agentur für Arbeit nicht \ndienstbereit ist (z. 

In [208]:
response_metadata

{'ref_0': {'page': '15',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_1': {'page': '6', 'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_2': {'page': '13',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_3': {'page': '14',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_4': {'page': '18',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}}

In [209]:
print(response_metadata_message)

There 5 sources:
---ref_0Page 15 from file merkblatt-fuer-arbeitslose_ba036520.pdf
---ref_1Page 6 from file merkblatt-fuer-arbeitslose_ba036520.pdf
---ref_2Page 13 from file merkblatt-fuer-arbeitslose_ba036520.pdf
---ref_3Page 14 from file merkblatt-fuer-arbeitslose_ba036520.pdf
---ref_4Page 18 from file merkblatt-fuer-arbeitslose_ba036520.pdf


In [210]:
eval_response(response_baseline)

Sie müssen sich spätestens am ersten Tag Ihrer Arbeitslosigkeit arbeitslos melden, um Arbeitslosengeld zu beantragen.

{'ref_0': {'page': '15',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_1': {'page': '6', 'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_2': {'page': '13',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_3': {'page': '14',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_4': {'page': '18',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}}

## Cosine Similarity Postprocessor

In [211]:
for node in response_baseline.source_nodes:
    cosine_similarity = node.score
    print(f"Cosine Similarity: {cosine_similarity}")

Cosine Similarity: 0.888295093393664
Cosine Similarity: 0.8882828878764354
Cosine Similarity: 0.8874749686893485
Cosine Similarity: 0.8834858040005298
Cosine Similarity: 0.8807909649499002


In [212]:
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.821)
postprocessed_nodes = postprocessor.postprocess_nodes(response_baseline.source_nodes)

for node in postprocessed_nodes:
    print(f'Page: {node.node.metadata["page_label"]}, Document: {node.node.metadata["file_name"]}, Cosine Similarity: {node.score:.2f}')

Page: 15, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.89
Page: 6, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.89
Page: 13, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.89
Page: 14, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.88
Page: 18, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.88


## SVM

In [213]:
query_modes = [
    "svm",
    "linear_regression",
    "logistic_regression",
]

responses_modes = []

for query_mode in query_modes:
    query_engine_modes = index_loaded.as_query_engine(
        text_qa_template=qa_template,
        similarity_top_k=number_top_results,
        vector_store_query_mode=query_mode)

    response_modes = query_engine_modes.query(user_query)
    responses_modes.append(response_modes)

/home/daniele/Desktop/Projects/bureaubot/.venv_bureaubot/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/daniele/Desktop/Projects/bureaubot/.venv_bureaubot/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/daniele/Desktop/Projects/bureaubot/.venv_bureaubot/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [214]:
for query_mode, r in zip(query_modes, responses_modes):
    print(f"Query mode: {query_mode}\n")
    print_response_info(user_query, r)
    print("="*90)

Query mode: svm

Prompt:
Wann muss ich mich melden wenn ich Arbeitslosengeld beantragen will?


Response:<br/>Sie müssen sich spätestens am ersten Tag Ihrer Arbeitslosigkeit arbeitslos melden, um Arbeitslosengeld zu beantragen.

Sources:
Page: 13, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.40
Page: 6, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.46
Page: 14, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.48
Page: 24, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.49
Page: 15, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.49
Query mode: linear_regression

Prompt:
Wann muss ich mich melden wenn ich Arbeitslosengeld beantragen will?


Response:<br/>Sie müssen sich spätestens am ersten Tag Ihrer Arbeitslosigkeit arbeitslos melden, um Arbeitslosengeld zu beantragen.

Sources:
Page: 13, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.40
Page: 6, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.46
Page: 14, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.48
Page: 24, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.49
Page: 15, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.49
Query mode: logistic_regression

Prompt:
Wann muss ich mich melden wenn ich Arbeitslosengeld beantragen will?


Response:<br/>Sie müssen sich spätestens am ersten Tag Ihrer Arbeitslosigkeit arbeitslos melden, um Arbeitslosengeld zu beantragen.

Sources:
Page: 13, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.40
Page: 6, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.46
Page: 14, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.48
Page: 24, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.49
Page: 15, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: -0.49


## Hypothetical document embeddings (HyDE)

In [215]:
query_engine_hyde = index_loaded.as_query_engine(text_qa_template=qa_template, similarity_top_k=number_top_results)

# "HyDEQueryTransform" generates a hypothetical document and use it for embedding lookup.
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine_hyde, hyde)
response_hyde = hyde_query_engine.query(user_query)

print_response_info(user_query, response_hyde)

Prompt:
Wann muss ich mich melden wenn ich Arbeitslosengeld beantragen will?


Response:<br/>Sie müssen sich spätestens am 1. Tag der Arbeitslosigkeit arbeitslos melden, um Arbeitslosengeld beantragen zu können.

Sources:
Page: 14, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.93
Page: 15, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.93
Page: 6, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.93
Page: 13, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.93
Page: 18, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.92


In [216]:
hyde

In [217]:
#In this example, HyDE improves output quality significantly, by hallucinating
# accurately, and thus improving the embedding quality, and final output.
query_bundle = hyde(user_query)
hyde_doc = query_bundle.embedding_strs[0]

print(f"Prompt:\n{user_query}\n\nResponse:")
display(Markdown(f"{hyde_doc}"))

Prompt:
Wann muss ich mich melden wenn ich Arbeitslosengeld beantragen will?

Response:


Um Arbeitslosengeld zu beantragen, müssen Sie sich so früh wie möglich bei der Agentur für Arbeit melden. Es wird empfohlen, dies spätestens drei Monate vor dem geplanten Beginn der Arbeitslosigkeit zu tun. Sie können sich persönlich bei der örtlichen Agentur für Arbeit anmelden oder dies online über die Website der Agentur tun. Bei der Anmeldung müssen Sie verschiedene Unterlagen vorlegen, wie zum Beispiel Ihren Personalausweis, Ihre Sozialversicherungsnummer, Ihren Lebenslauf, Ihre Arbeitsverträge der letzten zwei Jahre, Ihre Kündigungsschreiben und Ihre Bankverbindung. Es ist wichtig, dass Sie alle erforderlichen Dokumente vollständig und korrekt vorlegen, um Verzögerungen bei der Bearbeitung Ihres Antrags zu vermeiden. Sobald Sie sich angemeldet haben, wird die Agentur für Arbeit Ihren Anspruch auf Arbeitslosengeld prüfen und Ihnen gegebenenfalls weitere Informationen und Anweisungen geben. Es ist wichtig, dass Sie während des gesamten Prozesses eng mit der Agentur für Arbeit zusammenarbeiten und alle erforderlichen Informationen bereitstellen, um sicherzustellen, dass Ihr Antrag reibungslos bearbeitet wird.

In [218]:
hyde_2 = HyDEQueryTransform(include_original=True)
query_bundle_2 = hyde_2(user_query)
hyde_doc_2 = query_bundle_2.embedding_strs[0]

print(f"Prompt:\n{user_query}\n\nResponse:")
display(Markdown(f"{hyde_doc_2}"))

Prompt:
Wann muss ich mich melden wenn ich Arbeitslosengeld beantragen will?

Response:


Um Arbeitslosengeld zu beantragen, müssen Sie sich so früh wie möglich bei der Agentur für Arbeit melden. Es wird empfohlen, dies spätestens drei Monate vor dem geplanten Beginn der Arbeitslosigkeit zu tun. Sie können sich persönlich bei der örtlichen Agentur für Arbeit anmelden oder dies online über die Website der Agentur tun. Bei der Anmeldung müssen Sie verschiedene Unterlagen vorlegen, wie zum Beispiel Ihren Personalausweis, Ihre Sozialversicherungsnummer, Ihren Lebenslauf, Ihre Arbeitsverträge der letzten zwei Jahre und Ihre Kündigungserklärung. Es ist wichtig, dass Sie alle erforderlichen Unterlagen vollständig und korrekt vorlegen, um Verzögerungen bei der Bearbeitung Ihres Antrags zu vermeiden. Sobald Sie sich angemeldet haben, wird die Agentur für Arbeit Ihren Anspruch auf Arbeitslosengeld prüfen und Ihnen gegebenenfalls weitere Informationen und Anweisungen geben. Es ist wichtig, dass Sie während des gesamten Antragsprozesses erreichbar sind und auf Anfragen der Agentur für Arbeit reagieren, um sicherzustellen, dass Ihr Antrag reibungslos bearbeitet werden kann.

# Chat Engine

In [219]:
# Configure prompt parameters and initialise helper
max_input_size = 4096
num_output = 256
max_chunk_overlap = 0.2
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

system_prompt = (
    """
    You are an expert on the German administration system and your job is to answer technical questions.
    Assume that all questions are related to the the provided context.
    Keep your answers based on facts, do not hallucinate information.
    """
)

llm= LLMPredictor(llm=OpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",
    system_prompt=system_prompt
))

service_context = ServiceContext.from_defaults(llm_predictor=llm, prompt_helper=prompt_helper)

chat_engine = index_loaded.as_chat_engine(
    #service_context=service_context,
    chat_mode="context",
    # verbose=True
    similarity_top_k=3
)

In [220]:
response_chat_test = chat_engine.chat("What do you know about the city of Berlin?")
# response_chat_test = chat_engine.chat("Is the unemployment benefit based on the number of children I have?")

In [221]:
response_chat_test

AgentChatResponse(response="Berlin is the capital and largest city of Germany. It is located in the northeastern part of the country and is known for its rich history, vibrant culture, and diverse population. Here are some key points about Berlin:\n\n1. History: Berlin has a significant historical background, including its division during the Cold War when it was split into East Berlin (under Soviet control) and West Berlin (controlled by the Allies). The Berlin Wall, which separated the two parts, became a symbol of the Cold War. The city was reunified in 1990 after the fall of the wall.\n\n2. Landmarks: Berlin is home to numerous iconic landmarks, such as the Brandenburg Gate, a symbol of German unity; the Reichstag building, which houses the German parliament; the Berlin Wall Memorial; Checkpoint Charlie, a former border crossing point; and the Berlin TV Tower, offering panoramic views of the city.\n\n3. Cultural Hub: Berlin is renowned for its thriving arts and cultural scene. It h

In [222]:
metadata_chat = eval_response(response_chat_test)

metadata_chat

NO
info not available


In [223]:
print(f"There {len(response_chat_test.source_nodes)} sources.")
for source_node in response_chat_test.source_nodes:
    print(f'Page {source_node.node.metadata["page_label"]} from file {source_node.node.metadata["file_name"]}')

There 3 sources.
Page 31 from file dok_ba013155.pdf
Page 10 from file merkblatt-11-berufsberatung_ba015370.pdf
Page 16 from file merkblatt-algii_ba015397.pdf


In [224]:
response_chat_test.response

"Berlin is the capital and largest city of Germany. It is located in the northeastern part of the country and is known for its rich history, vibrant culture, and diverse population. Here are some key points about Berlin:\n\n1. History: Berlin has a significant historical background, including its division during the Cold War when it was split into East Berlin (under Soviet control) and West Berlin (controlled by the Allies). The Berlin Wall, which separated the two parts, became a symbol of the Cold War. The city was reunified in 1990 after the fall of the wall.\n\n2. Landmarks: Berlin is home to numerous iconic landmarks, such as the Brandenburg Gate, a symbol of German unity; the Reichstag building, which houses the German parliament; the Berlin Wall Memorial; Checkpoint Charlie, a former border crossing point; and the Berlin TV Tower, offering panoramic views of the city.\n\n3. Cultural Hub: Berlin is renowned for its thriving arts and cultural scene. It has over 180 museums, includ

In [225]:
len(response_chat_test.source_nodes)

3

In [226]:
response_chat_test.source_nodes[0]

NodeWithScore(node=TextNode(id_='101c5b70-98d4-432f-bfd5-80811b2128f7', embedding=None, metadata={'page_label': '31', 'file_name': 'dok_ba013155.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='360b1b3b-dfeb-455b-8a57-9e29ae2036f5', node_type=None, metadata={'page_label': '31', 'file_name': 'dok_ba013155.pdf'}, hash='e480367de126c1d8d7861417949102d1ec5d2e2d2f802eec483440253fd1a247')}, hash='e480367de126c1d8d7861417949102d1ec5d2e2d2f802eec483440253fd1a247', text='315 In Deutschland  gearbeitet und im Ausland gewohnt\n5  Sie haben in Deutschland \n gearbeitet und haben als \nGrenzgängerin bzw. Grenz -\ngänger im (benach  barten) \nAusland gewohnt?\n5.1  Zusätzliche Arbeitsuchendmeldung von \nGrenzgängerinnen bzw. Grenzgängern im \nbisherigen Beschäftigungsstaat\nWenn Sie in einem anderen Mitgliedstaat wohnen und \neine Beschäftigung von dort aus als Grenzgängerin \nbzw. Grenzgänger in Deutschlan

In [227]:
response_chat_test.source_nodes[0].node.metadata["page_label"]

'31'